In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import gamma
from scipy.special import erf
from scipy.signal import welch
import warnings
warnings.filterwarnings(action='ignore');
import rainflow
from scipy.signal import lfilter

In [2]:
def fileO(x,y,filename):
    f = open(filename,'w');
    #f = open(filename,'a'); # overwrite
    for i in range(len(y)):
        f.write('%.4e\t%.4e\n'%(x[i],y[i]));
    f.close();

In [3]:
def fileI(filename):
    tmp = []
    f = open(filename,'r');
    lines = f.readlines()
    for line in lines:
        tmp.append(line);
    out=np.zeros([len(tmp),2]);
    for i in range(len(out)):
        out[i,:] = np.array(tmp[i].split()).astype(np.float)
    return out;

In [4]:
def log_interp(x,x_spec,y_spec):
    return 10**np.interp(np.log10(x),np.log10(x_spec),np.log10(y_spec));

In [5]:
def blwn(f,fmin,fmax,bwp,loc):  # bwp : band width percentage # loc : location percentage
    df = (fmax-fmin)*bwp;
    f1 = fmin + loc*(fmax-fmin-df);
    f2 = f1+df;
    A = (10**2) / (f2-f1)    # grms = 10
    if f >= f1 and f <= f2:
        out = A;
    else:
        out = 0;
    return out;

In [6]:
def unimodal(f,fmin,fmax,bwp,loc,fn1,zeta1):
    df = (fmax-fmin)*bwp;
    f1 = fmin + loc*(fmax-fmin-df);
    f2 = f1+df;
    f_tmp = np.linspace(f1,f2,10000);
    A = 1;
    B = fn1**4 + (2*zeta1*f_tmp*fn1)**2;
    C = (fn1**2-f_tmp**2)**2 + (2*zeta1*f_tmp*fn1)**2;
    out = A*B/C;
    grms_tmp = np.sqrt(np.trapz(out,f_tmp));
    A = (10/grms_tmp)**2             # grms = 10
    B = fn1**4 + (2*zeta1*f*fn1)**2;
    C = (fn1**2-f**2)**2 + (2*zeta1*f*fn1)**2;
    if f >= f1 and f <= f2:
        out = out = A*B/C;
    else:
        out = 0;
    return out;

In [7]:
def bimodal(f,fmin,fmax,bwp,loc,fn1,zeta1,fn2,zeta2,ratio):
    df = (fmax-fmin)*bwp;
    f1 = fmin + loc*(fmax-fmin-df);
    f2 = f1+df;
    f_tmp = np.linspace(f1,f2,10000);
    A1 = 1
    B1 = fn1**4 + (2*zeta1*f_tmp*fn1)**2;
    C1 = (fn1**2-f_tmp**2)**2 + (2*zeta1*f_tmp*fn1)**2;
    A2 = ratio;
    B2 = fn2**4 + (2*zeta2*f_tmp*fn2)**2;
    C2 = (fn2**2-f_tmp**2)**2 + (2*zeta2*f_tmp*fn2)**2;
    out = A1*B1/C1 + A2*B2/C2;
    grms_tmp = np.sqrt(np.trapz(out,f_tmp));
    A1 *= (10/grms_tmp)**2             # grms = 10  
    B1 = fn1**4 + (2*zeta1*f*fn1)**2;
    C1 = (fn1**2-f**2)**2 + (2*zeta1*f*fn1)**2;
    A2 *= (10/grms_tmp)**2             # grms = 10
    B2 = fn2**4 + (2*zeta2*f*fn2)**2;
    C2 = (fn2**2-f**2)**2 + (2*zeta2*f*fn2)**2;   
    if f >= f1 and f <= f2:
        out = A1*B1/C1 + A2*B2/C2;
    else:
        out = 0;
    return out;

In [8]:
def M(freq,psd,n):
    psd1 = psd*freq**n;
    return np.trapz(psd1,freq);

In [9]:
def D_NB(freq,psd,m,T,C):
    M0 = M(freq,psd,0);  M2 = M(freq,psd,2);
    E0 = np.sqrt(M2/M0);
    return E0*T*((2*M0)**(m/2))*gamma(m/2+1) / C;

In [ ]:
def D_NB1(freq,psd,m,T,C):                # for Single Moment method
    k = 2.0 / m;
    Mk = M(freq,psd,k);
    return (Mk**(m/2))*T*(2**(m/2))*gamma(m/2+1) / C;

In [10]:
def D_WL(freq,psd,m,T,C):
    DNB = D_NB(freq,psd,m,T,C);
    M0 = M(freq,psd,0);  M2 = M(freq,psd,2);  M4 = M(freq,psd,4);
    E0 = np.sqrt(M2/M0);
    Ep = np.sqrt(M4/M2);
    r = E0/Ep;
    lam = np.sqrt(1-r**2);
    a = 0.926 - 0.033*m;
    b = 1.587*m - 2.323;
    zeta = a + (1-a)*(1-lam)**b
    return DNB*zeta;

In [11]:
def D_OC(freq,psd,m,T,C):
    DNB = D_NB(freq,psd,m,T,C);
    k = 2/m;
    M0 = M(freq,psd,0);  M2 = M(freq,psd,2);  M4 = M(freq,psd,4); Mk = M(freq,psd,k); Mk2 = M(freq,psd,k+2);
    #print("M0=%.2e, M2=%.2e, M4=%.2e"%(M0,M2,M4));
    E0 = np.sqrt(M2/M0);
    Ep = np.sqrt(M4/M2);
    r = E0/Ep;
    zeta = (1/r)*((M2*Mk)/(M0*Mk2))**(m/2);
    return DNB*zeta;

In [ ]:
def D_SM(freq,psd,m,T,C):   # Larsen and Lutes' Single Moment (SM) Method
    DNB = D_NB1(freq,psd,m,T,C);
    k = 2/m;
    M0 = M(freq,psd,0);  M2 = M(freq,psd,2);  M4 = M(freq,psd,4); Mk = M(freq,psd,k); Mk2 = M(freq,psd,k+2);
    E0 = np.sqrt(M2/M0);
    Ep = np.sqrt(M4/M2);
    zeta = ((Mk/M0)**(m/2)) / E0;
    return DNB*zeta;

In [ ]:
def D_a75(freq,psd,m,T,C):  # Benascuitti and Tovo's alpha.75 MEthod
    DNB = D_NB(freq,psd,m,T,C);
    M75 = M(freq,psd,0.75);  M0 = M(freq,psd,0);  M275 = M(freq,psd,2*0.75);
    a75 = np.sqrt(M75/(M0*M275));
    zeta = a75**2;
    #print(DNB);
    #print("M0=%.2e, M75=%.2e, M275=%.2e"%(M0,M75,M275));
    #print("a75=%.2e"%(a75));
    return DNB*zeta;

In [ ]:
def D_BT(freq,psd,m,T,C):
    DNB = D_NB(freq,psd,m,T,C);
    M0 = M(freq,psd,0);  M1 = M(freq,psd,1);  M2 = M(freq,psd,2);  M4 = M(freq,psd,4);
    a1 = M1/np.sqrt(M0*M2);  a2 = M2/np.sqrt(M0*M4);
    b = (a1-a2)*(1.112*(1+a1*a2-(a1+a2))*np.exp(2.11*a2)+(a1-a2)) / ((a2-1)**2)
    zeta = b + (1-b)*(a2**(m-1));
    return DNB*zeta;

In [12]:
def D_dirlik(freq,psd,m,T,C):
    M0 = M(freq,psd,0);  M1 = M(freq,psd,1);  M2 = M(freq,psd,2);  M4 = M(freq,psd,4);
    E0 = np.sqrt(M2/M0);
    Ep = np.sqrt(M4/M2);
    r = E0/Ep;    
    Xm = (M1/M0)*np.sqrt(M2/M4);
    Z = 1/np.sqrt(M0);
    D1 = 2*(Xm-r**2)/(1+r**2);
    R = (r-Xm-D1**2)/(1-r-D1+D1**2);
    D2 = (1-r-D1+D1**2)/(1-R);
    D3 = 1 - D1 - D2;
    Q = 1.25*(r-D3-D2*R)/D1
    def f(sa):
        A1 = D1/(np.sqrt(M0)*Q);
        A2 = D2*Z*sa/(np.sqrt(M0)*R**2);
        A3 = D3*Z*sa/np.sqrt(M0);
        a1 = -Z*sa/Q
        a2 = -(Z**2)*(sa**2)/(2*R**2);
        a3 = -(Z**2)*(sa**2)/2;
        return A1*np.exp(a1) + A2*np.exp(a2) + A3*np.exp(a3);
    def z(m):
        mm = np.linspace(3,12,10);
        zz = np.array([8.652,8.822,8.982,9.133,9.277,9.415,9.546,9.673,9.796,9.915]);
        return np.interp(m,mm,zz);
    LB = 0; UB = z(m)*np.sqrt(M0);
    s = np.linspace(LB,UB,10000);
    samfsa = (s**m)*f(s);
    intsamfsa = np.trapz(samfsa,s);
    return Ep*T*intsamfsa/C;

In [13]:
def H(f,fn,zeta):   # Input : Acceleration / Output : Relative Displacement
    return 386.4/(((2*np.pi)**2)*(fn**2-f**2+1j*2*zeta*f*fn));

In [14]:
def HH(f,fn,zeta):   # Input : Acceleration / Output : Relative Displacement
    return np.abs(H(f,fn,zeta)*np.conjugate(H(f,fn,zeta)));

In [15]:
def FDS_dirlik(fi,pi,zeta,m,T,C): 
    fn = np.logspace(0,3,5000);
    D = np.zeros(len(fn));
    for i in range(len(fn)):
        K = 1
        rpsd = pi*HH(fi,fn[i],zeta)*K;
        D[i] = D_dirlik(fi,rpsd,m=m,T=1,C=1);
    return fn,D;

In [16]:
def FDS(fi,pi,zeta,m,T,C,model): 
    #fn = np.logspace(0,3,5000);
    fn=np.linspace(1,1000,5000);
    D = np.zeros(len(fn));
    for i in range(len(fn)):
        K = 1
        rpsd = pi*HH(fi,fn[i],zeta)*K;
        if model == 1:
            D[i] = D_OC(fi,rpsd,m=m,T=T,C=1);
        if model == 2:
            D[i] = D_SM(fi,rpsd,m=m,T=T,C=1);
        if model == 3:
            D[i] = D_a75(fi,rpsd,m=m,T=T,C=1);
        if model == 4:
            D[i] = D_dirlik(fi,rpsd,m=m,T=T,C=1);
        if model == 5:
            D[i] = D_BT(fi,rpsd,m=m,T=T,C=1);
    return fn,D;

In [17]:
def band_split(fi,pi,nb): # fi : interpolated freq, ai : interpolated psd, nb : # of bands
    ni = len(fi);
    grms = np.sqrt(np.trapz(pi,fi))
    goal = grms/np.sqrt(nb);  # target grms
    idx = np.zeros(nb+1,'int');
    grms_tmp = 0;
    for i in range(1,nb):
        while(grms_tmp < goal):
            idx[i] += 1;
            grms_tmp = np.sqrt(np.trapz(pi[idx[i-1]:idx[i]],fi[idx[i-1]:idx[i]]));
        grms_tmp1 = np.sqrt(np.trapz(pi[idx[i-1]:idx[i]-1],fi[idx[i-1]:idx[i]-1]));
        if np.abs(grms_tmp1 - goal) < np.abs(grms_tmp - goal):
            idx[i] += 1;
        grms_tmp = 0; grms_tmp1 = 0;
    idx[0] = 0; idx[nb] = len(pi);
    dfi = []; dpi = [];
    for i in range(nb):
        dfi.append(fi[idx[i]:idx[i+1]+1]); dpi.append(pi[idx[i]:idx[i+1]+1]);
    return dfi,dpi

In [18]:
def bs_overlap(dfi,dpi,p): # 각 band의 데이터 수 x p(%) 만큼 overlap
    nb = len(dpi);
    ndata, pndata = np.zeros(nb,int),np.zeros(nb,int);
    dfi1 = np.copy(dfi); dpi1 = np.copy(dpi);
    for i in range(nb):
        ndata[i] = len(dpi[i]);
        pndata[i] = int(ndata[i]*p);
    for i in range(nb-1):
        dfi1[i] = np.concatenate([dfi[i],dfi[i+1][1:pndata[i+1]+1]]);
        dpi1[i] = np.concatenate([dpi[i],dpi[i+1][1:pndata[i+1]+1]]);
    for i in range(1,nb):        
        dfi1[i] = np.concatenate([dfi[i-1][-pndata[i]:1],dfi1[i]]);
        dpi1[i] = np.concatenate([dpi[i-1][-pndata[i]:1],dpi1[i]]);     
    return dfi1,dpi1

In [21]:
def blwn_test_fd(fmin,fmax,bwp,loc,z,mm,amp,ovl,model,dur):  # frequency domain
    # Generate Input PSD
    #freq_spec = np.logspace(np.log10(fmin),np.log10(fmax),5000); psd_spec = np.zeros(5000); 
    freq_spec=np.linspace(1,1000,5000); psd_spec=np.zeros(5000);
    for i in range(5000):
        psd_spec[i]= blwn(freq_spec[i],fmin,fmax,bwp,loc);
    # Band Split
    fi,pi = freq_spec,psd_spec;
    dfi,dpi = band_split(fi,pi,2);
    if ovl > 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
        dfi,dpi = bs_overlap(dfi,dpi,ovl); # overlap
    # for i in range(2):
        # print('freq range : %.2f Hz ~ %.2f Hz, grms%d = %.4f'%(min(dfi[i]),max(dfi[i]),i+1,np.sqrt(np.trapz(dpi[i],dfi[i]))));
    # Calculate FDS
    fn,D  = FDS(fi    ,pi      ,zeta=z,m=mm,T=dur,C=1,model=model);
    fn,D1 = FDS(dfi[0],dpi[0]*amp,zeta=z,m=mm,T=dur,C=1,model=model);
    fn,D2 = FDS(dfi[1],dpi[1]*amp,zeta=z,m=mm,T=dur,C=1,model=model);

    

   ## Plot
   #plt.figure(figsize=(16,16));
   #plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
   #plt.subplot(321);
   #plt.plot(dfi[0],dpi[0],label='Band1'); plt.plot(dfi[1],dpi[1],label='Band2'); plt.legend();
   #plt.ylim(max(dpi[0])/100,max(dpi[0])*100);
   ##plt.xscale('log'); 
   #plt.yscale('log'); plt.xlim(fmin,fmax); plt.title('Input PSD(BW=%.1f, LOC=%.1f)'%(bwp,loc)); plt.xlabel('Frequency(Hz)'); plt.ylabel('PSD(g^2/Hz)');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(322);
   #plt.plot(fn,D,label='Original'); plt.plot(fn,D1,label='Band1'); plt.plot(fn,D2,label='Band2'); 
   ##plt.xscale('log'); 
   #plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
   #plt.title('FDS'); plt.xlabel('Natural Frequency(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(323);
   #if ovl == 0:
   #    xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
   #ylim1 = min(D[(fn>xlim1)&(fn<xlim2)])*0.1; 
   #ylim2 = max(D[(fn>xlim1)&(fn<xlim2)])*10; 
   #plt.plot(fn,D,'k-',label='Original'); plt.plot(fn,D1,'r--',label='Band1'); plt.plot(fn,D2,'b--',label='Band2'); 
   #plt.plot(fn,D1+D2,'g--',label='split(sum)');  plt.yscale('log');
   #plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
   #plt.title('FDS, Near Band Split Frequency');
   #plt.grid(b=True, which='major', linestyle='-');
   #plt.subplot(324);
   #plt.plot(fn,D,'k-',label='Original'); plt.plot(fn,D1+D2,'r--',label='Sum of Split Bands');  
   ##plt.xscale('log'); 
   #plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
   #plt.title('FDS'); plt.xlabel('Natural Frequency(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(325);
    rD = D/(D1+D2);
   #plt.plot(fn,rD,'k-',label='FDS Ratio (Original/Sum of Split Bands)');
   #plt.legend(); plt.xlim(fmin,fmax);
   ##plt.xscale('log'); 
   #plt.title('FDS Ratio (Original/Sum of Split Bands)'%(max(rD))); plt.xlabel('Natural Frequency(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(326);
   #plt.plot(fn,rD,'k-',label='Original/Sum of Split Bands');
   #ylim1 = min(rD[(fn>xlim1)&(fn<xlim2)])*0.5; ylim2 = max(rD[(fn>xlim1)&(fn<xlim2)])*2;
   #plt.grid(which='both'); plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
   #plt.title('FDS Ratio (Original/Sum of Split Bands), Near Band Split Frequency'); plt.xlabel('Natural Frequency(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');\
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.savefig('fds/blwn/blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_fd.png'%(bwp,loc,z,mm,amp,ovl), dpi=300);  
   #plt.close();
 
    #fileO(fn,rD,'fds/blwn/blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_fd_rD.txt'%(bwp,loc,z,mm,amp,ovl));
   #fileO(fn,D,'fds/blwn/blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_fd_FDS1.txt'%(bwp,loc,z,mm,amp,ovl));
   #fileO(fn,D1,'fds/blwn/blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_fd_FDS2.txt'%(bwp,loc,z,mm,amp,ovl));
   #fileO(fn,D2,'fds/blwn/blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_fd_FDS3.txt'%(bwp,loc,z,mm,amp,ovl));

    f = open("fds/blwn/summary.txt",'a');
    f.write('%.1fb %.1fl %.2fz %dm %.2fa %.2fo rD max = %.2f \n'%(bwp,loc,z,mm,amp,ovl,max(rD)));
    f.close();
    return fn,rD;

In [ ]:
def unimodal_test_fd(fmin,fmax,bwp,loc,fn1,zeta1,z,mm,amp,ovl,model,dur): # frequency domain
    # Generate Input PSD
    freq_spec = np.logspace(np.log10(fmin),np.log10(fmax),5000); psd_spec = np.zeros(5000); 
    for i in range(5000):
        psd_spec[i]= unimodal(freq_spec[i],fmin,fmax,bwp,loc,fn1,zeta1);
    # Band Split
    fi,pi = freq_spec,psd_spec;
    dfi,dpi = band_split(fi,pi,2);
    if ovl > 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
        dfi,dpi = bs_overlap(dfi,dpi,ovl); # overlap    
    # for i in range(2):
        # print('freq range : %.2f Hz ~ %.2f Hz, grms%d = %.4f'%(min(dfi[i]),max(dfi[i]),i+1,np.sqrt(np.trapz(dpi[i],dfi[i]))));
    # Calculate FDS
    fn,D  = FDS(fi    ,pi      ,zeta=z,m=mm,T=dur,C=1,model=model);
    fn,D1 = FDS(dfi[0],dpi[0]*amp,zeta=z,m=mm,T=dur,C=1,model=model);
    fn,D2 = FDS(dfi[1],dpi[1]*amp,zeta=z,m=mm,T=dur,C=1,model=model);
   ## Plot
   #plt.figure(figsize=(16,16));
   #plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
   #plt.subplot(321);
   #plt.plot(dfi[0],dpi[0],label='band1'); plt.plot(dfi[1],dpi[1],label='band2'); plt.legend();
   ##plt.xscale('log'); 
   #plt.yscale('log'); plt.xlim(fmin,fmax); plt.title('Input PSD(BW=%.1f, LOC=%.1f, fn1 = %d, zeta1 = %.2f)'%(bwp,loc,fn1,zeta1)); plt.xlabel('Freq(Hz)'); plt.ylabel('PSD(g^2/Hz)');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(322);
   #plt.plot(fn,D,label='original'); plt.plot(fn,D1,label='band1'); plt.plot(fn,D2,label='band2'); 
   ##plt.xscale('log'); 
   #plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
   #plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(323);
   #if ovl == 0:
   #    xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
   #ylim1 = min(D[(fn>xlim1)&(fn<xlim2)])*0.1; 
   #ylim2 = max(D[(fn>xlim1)&(fn<xlim2)])*10; 
   #plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1,'r--',label='band1'); plt.plot(fn,D2,'b--',label='band2'); 
   #plt.plot(fn,D1+D2,'g--',label='split(sum)');  plt.yscale('log');
   #plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
   #plt.title('FDS, Near Band Split Freq');
   #plt.grid(b=True, which='major', linestyle='-');
   #plt.subplot(324);
   #plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1+D2,'r--',label='split(sum)');  
   ##plt.xscale('log');
   #plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
   #plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(325);
    rD = D/(D1+D2);
   #plt.plot(fn,rD,'k-',label='original/split(sum)');
   #plt.legend(); plt.xlim(fmin,fmax);
   ##plt.xscale('log'); 
   #plt.title('FDS Ratio(original/split(sum))'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.subplot(326);
   #plt.plot(fn,rD,'k-',label='original/split(sum)');
   #ylim1 = min(rD[(fn>xlim1)&(fn<xlim2)])*0.5; ylim2 = max(rD[(fn>xlim1)&(fn<xlim2)])*2;
   #plt.grid(which='both'); plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
   #plt.title('FDS Ratio(original/split(sum)), Near Band Split Freq'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');\
   #plt.grid(b=True, which='both', linestyle='-');
   #plt.savefig('fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_fd.png'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl), dpi=300);   # Bandwidth, Location, 1% damping FDS, m = 4
   #plt.close();
    
   #fileO(fn,rD,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_fd_rD.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
   #fileO(fn,D,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_fd_FDS1.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
   #fileO(fn,D1,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_fd_FDS2.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
   #fileO(fn,D2,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_fd_FDS3.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
    
    f = open("fds/unimodal/summary.txt",'a');
    f.write('%.1fb %.1fl %.2ff1 %.2fz1 %.2fz %dm %.2fa %.2fo rD max = %.2f \n'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl,max(rD)));
    f.close();
    return fn,rD;

In [ ]:
def bimodal_test_fd(fmin,fmax,bwp,loc,fn1,zeta1,fn2,zeta2,ratio,z,mm,amp,ovl,model,dur): # frequency domain
    # Generate Input PSD
    freq_spec = np.logspace(np.log10(fmin),np.log10(fmax),5000); psd_spec = np.zeros(5000); 
    for i in range(5000):
        psd_spec[i]= bimodal(freq_spec[i],fmin,fmax,bwp,loc,fn1,zeta1,fn2,zeta2,ratio);
    # Band Split
    fi,pi = freq_spec,psd_spec;
    dfi,dpi = band_split(fi,pi,2);
    if ovl > 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
        dfi,dpi = bs_overlap(dfi,dpi,ovl); # overlap  
    # for i in range(2):
        # print('freq range : %.2f Hz ~ %.2f Hz, grms%d = %.4f'%(min(dfi[i]),max(dfi[i]),i+1,np.sqrt(np.trapz(dpi[i],dfi[i]))));
    # Calculate FDS
    fn,D  = FDS(fi    ,pi      ,zeta=z,m=mm,T=dur,C=1,model=model);
    fn,D1 = FDS(dfi[0],dpi[0]*amp,zeta=z,m=mm,T=dur,C=1,model=model);
    fn,D2 = FDS(dfi[1],dpi[1]*amp,zeta=z,m=mm,T=dur,C=1,model=model);
    ## Plot
    #plt.figure(figsize=(16,16));
    #plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
    #plt.subplot(321);
    #plt.plot(dfi[0],dpi[0],label='band1'); plt.plot(dfi[1],dpi[1],label='band2'); plt.legend();
    ##plt.xscale('log'); 
    #plt.yscale('log'); plt.xlim(fmin,fmax); plt.title('Input PSD(BW=%.1f, LOC=%.1f, fn1 = %d, zeta1 = %.2f, fn2 = %d, zeta2 = %.2f, ratio = %.2f)'%(bwp,loc,fn1,zeta1,fn2,zeta2,ratio)); plt.xlabel('Freq(Hz)'); plt.ylabel('PSD(g^2/Hz)');
    #plt.grid(b=True, which='both', linestyle='-');
    #plt.subplot(322);
    #plt.plot(fn,D,label='original'); plt.plot(fn,D1,label='band1'); plt.plot(fn,D2,label='band2'); 
    ##plt.xscale('log'); 
    #plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    #plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    #plt.grid(b=True, which='both', linestyle='-');
    #plt.subplot(323);
    #if ovl == 0:
    #    xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
    #ylim1 = min(D[(fn>xlim1)&(fn<xlim2)])*0.1; 
    #ylim2 = max(D[(fn>xlim1)&(fn<xlim2)])*10; 
    #plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1,'r--',label='band1'); plt.plot(fn,D2,'b--',label='band2'); 
    #plt.plot(fn,D1+D2,'g--',label='split(sum)');  plt.yscale('log');
    #plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    #plt.title('FDS, Near Band Split Freq');
    #plt.grid(b=True, which='major', linestyle='-');
    #plt.subplot(324);
    #plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1+D2,'r--',label='split(sum)');  
    ##plt.xscale('log'); 
    #plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    #plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    #plt.grid(b=True, which='both', linestyle='-');
    #plt.subplot(325);
    rD = D/(D1+D2);
    #plt.plot(fn,rD,'k-',label='original/split(sum)');
    #plt.legend(); plt.xlim(fmin,fmax);
    ##plt.xscale('log'); 
    #plt.title('FDS Ratio(original/split(sum))'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');
    #plt.grid(b=True, which='both', linestyle='-');
    #plt.subplot(326);
    #plt.plot(fn,rD,'k-',label='original/split(sum)');
    #ylim1 = min(rD[(fn>xlim1)&(fn<xlim2)])*0.5; ylim2 = max(rD[(fn>xlim1)&(fn<xlim2)])*2;
    #plt.grid(which='both'); plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    #plt.title('FDS Ratio(original/split(sum)), Near Band Split Freq'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');\
    #plt.grid(b=True, which='both', linestyle='-');
    #plt.savefig('fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_fd.png'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl), dpi=300);   # Bandwidth, Location, 1% damping FDS, m = 4
    #plt.close();
    #
    #fileO(fn,rD,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_fd_rD.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
    #fileO(fn,D,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_fd_FDS1.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
    #fileO(fn,D1,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_fd_FDS2.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
    #fileO(fn,D2,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_fd_FDS3.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));

        
    f = open("fds/bimodal/summary.txt",'a');
    f.write('%.1fb %.1fl %.2ff1 %.2fz1 %.2ff2 %.2fz2 %.2fz %dm %.2fa %.2fo rD max = %.2f \n'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl,max(rD)));
    f.close();
    return fn,rD;

In [ ]:
def psd2th_wn(freq,psd,dur):
    def psdinteg(freq,psd):
        f = freq; y = psd;
        ndata = len(y);
        a = np.zeros(ndata-1);
        for i in range(ndata-1):
            if y[i] != 0 and y[i+1] != 0:
                n = np.log10(y[i+1]/y[i])/np.log10(f[i+1]/f[i]);
                if n == -1:
                    a[i] = y[i]*f[i]*np.log(f[i+1]/f[i]);
                else:
                    a[i] = (y[i]/(f[i]**n))*(1/(n+1))*(f[i+1]**(n+1)-f[i]**(n+1));
        return np.sqrt(sum(a))
    def log_interp(x,x_spec,y_spec):
        return 10**np.interp(np.log10(x),np.log10(x_spec),np.log10(y_spec), left=-10, right=-10);
    
    fmax = max(freq);
    tmax = dur;
    fs = fmax * 20.
    dt = 1 / fs

    spec_grms = psdinteg(freq,psd);

    Nt = int(np.ceil(tmax / dt))
    Nt3 = 3 * Nt
    
    # num_fft = 2의 n승
    Nt=2**int(np.ceil(np.log(Nt)/np.log(2)));  
    df = 1. / (Nt * dt);
    
    # White Noise
    wn = np.random.normal(0,1,Nt);
    
    Nf = int(Nt / 2)

    fft_freq = np.linspace(0, (Nf - 1) * df, Nf)
    fft_freq2 = np.linspace(0, (Nt - 1) * df, Nt)
    
    spec = np.zeros(Nf, float)
    sq_spec = np.zeros(Nf, float)
    
    js = 0
    
    if(fft_freq[0]<=0):
        fft_freq[0]=0.5*fft_freq[1];        
    
    sq_spec = np.sqrt(log_interp(fft_freq,freq,psd));
    
    Y = np.zeros(Nt, complex)

    YF = np.fft.fft(wn)
    
    YFn=YF[0:Nf]

    Y[0:Nf]=sq_spec*YFn

    Y[0]=0.
    
    for j in range(1, Nf):
        Y[Nt - j] = complex(Y[j].real, -Y[j].imag)
    
    YI = np.fft.ifft(Y)  
    
    psd_th = YI.real
    
    Nt = len(psd_th)
    
    time = np.linspace(0, (Nt - 1) * dt, Nt)
    
    syn_grms = np.std(psd_th)
    
    psd_th *= (spec_grms / syn_grms)
    
    return time,psd_th;

In [ ]:
def th2psd(time,th,sps):
    #sps = 2**12;
    dt = time[1]-time[0];
    freq_syn,psd_syn = welch(th,fs = 1/dt,nperseg=sps);
    return freq_syn,psd_syn;

In [ ]:
# lftab = freqeuncy table / ldtab = input time history / dr = damping ratio / dtime = integration time step(default = 1/(20*fmax))
# it = input type (0=displacement / 1=acceleration)

def th2fds(lftab,ldtab,dr,dtime,it,K,m,C):
    def newmark_parameter(dt):
        r = 0.005; alpha = (1/4)*((1+r)**2); delta = 0.5 + r;
        a = np.zeros(8);
        a[0] = 1/(alpha*dt*dt); a[1] = delta/(alpha*dt); a[2] = 1/(alpha*dt); a[3] = 1/(2*alpha)-1;
        a[4] = delta/alpha - 1; a[5] = (dt/2)*((delta/alpha)-2); a[6] = dt*(1-delta); a[7] = delta*dt;
        return a;
    def DERIV(X,Y):
        N = len(Y); YY = np.zeros(N);
        YY[0] = (Y[1]-Y[0])/(X[1]-X[0]); Y[N-1] = (Y[N-1]-Y[N-2])/(X[N-1]-X[N-2]);
        for n in range(N-2):
            YY[n+1] = (((Y[n+2]-Y[n+1])/(X[n+2]-X[n+1]))*(X[n+1]-X[n]) + ((Y[n+1]-Y[n])/(X[n+1]-X[n]))*(X[n+2]-X[n+1]))/(X[n+2]-X[n]);
        return YY;
    def INTEG(X,Y,C1):
        N = len(Y); YY = np.zeros(N);
        YY[0] = C1;
        for n in range(1,N):
            YY[n] = YY[n-1] + (1/2)*(Y[n]+Y[n-1])*(X[n]-X[n-1]);
        return YY;      
        
    freq = lftab; th = ldtab; zeta = dr; dt = dtime;
    a = newmark_parameter(dt);
    fds = np.zeros(len(freq));
    u,v,ac = np.zeros(len(th)),np.zeros(len(th)),np.zeros(len(th));
    ru,rv,rac = np.zeros(len(th)),np.zeros(len(th)),np.zeros(len(th));
    if it == 0:  # input type = displacement
        N = len(th);
        time = np.arange(N)*dt;
        vth = DERIV(time,th);       
        for fl in range(len(freq)):
            w = 2*np.pi*freq[fl]; m = 1; c = 2*zeta*w; k = w**2;
            F = 2*zeta*w*vth + w**2*th;
            Fe = a[0]*m + a[1]*c + k;
            
            for n in range(len(th-1)):
                u[n+1] = (F[n+1] + m*(a[0]*u[n]+a[2]*v[n]+a[3]*ac[n])+c*(a[1]*u[n]+a[4]*v[n]+a[5]*ac[n]))/Fe;
                v[n+1] = a[1]*(u[n+1]-u[n]) - a[4]*v[n] - a[5]*ac[n];
                ac[n+1] = a[0]*(u[n+1]-u[n]) -a[2]*v[n] - a[3]*ac[n];
            ru = u - th; rv = v - vth;
            st = ru*K;      # Stress Response
            rcc = rainflow.count_cycles(st); # rainflow counting
            rang, cycle = np.zeros(len(rcc)),np.zeros(len(rcc));
            for i in range(len(rcc)):
                rang[i] = rcc[i][0];
                cycle[i] = rcc[i][1];
            fds[fl] = sum((rang**m)*cycle)/C;
    if it == 1: # input type = acceleration
        N = len(th);
        time = np.arange(N)*dt;
        vth = INTEG(time,th,0); dth = INTEG(time,vth,0);
        F = -th;
        for fl in range(len(freq)):
            w = 2*np.pi*freq[fl]; m = 1; c = 2*zeta*w; k = w**2;            
            Fe = a[0]*m + a[1]*c + k;          
            for n in range(len(th)-1):
                ru[n+1] = (F[n+1] + m*(a[0]*ru[n]+a[2]*rv[n]+a[3]*rac[n])+c*(a[1]*ru[n]+a[4]*rv[n]+a[5]*rac[n]))/Fe;
                rv[n+1] = a[1]*(ru[n+1]-ru[n]) - a[4]*rv[n] - a[5]*rac[n];
                rac[n+1] = a[0]*(ru[n+1]-ru[n]) -a[2]*rv[n] - a[3]*rac[n];
            st = ru*K;      # Stress Response
            rcc = rainflow.count_cycles(st); # rainflow counting
            rang, cycle = np.zeros(len(rcc)),np.zeros(len(rcc));
            for i in range(len(rcc)):
                rang[i] = rcc[i][0];
                cycle[i] = rcc[i][1];
            fds[fl] = sum((rang**m)*cycle)/C;
    return freq,fds

In [ ]:
# f1,f2 = frequency range (fmin, fmax)
# oct = freqeucny gap (oct = 1./3.; oct = 1./6.; oct = 1./12.)
# iom = output option (1 = acceleration / 2 = relative displacement)
# irdu = output unit (1=inch / 2=mm)
# bex = fatigue exponent


def th2fds_tom(t1,th1,f1,f2,oct,iom,irdu,bex):
    dt = t1[1]-t1[0]; sr = 1/dt;
    a,b,num,dur=t1,th1,len(th1),max(t1);    
    damp = 0.05;
    Q = 1/(2*damp);
    nt=int(0.8*(1/f1)/dt);
    resp=np.zeros(num);  fn=np.zeros(num);
    fn[0]=f1    
    for j in range(1,999):
        fn[j]=fn[j-1]*(2.**oct)
        #if  fn[j] > sr/6. or fn[j] > f2:
        if  fn[j] > f2:
            break
    
    num_fn=j+1;
    
    temp=fn[0:num_fn]
    del(fn)
    fn=temp
    del(temp)
    
    pv_pos=np.zeros(num_fn)
    pv_neg=np.zeros(num_fn)
    rd_pos=np.zeros(num_fn)
    rd_neg=np.zeros(num_fn)
    def a_coeff(omega,damp,dt): 
        ac=np.zeros(3)    
        ac[0]=1   
        omegad=omega*np.sqrt(1.-(damp**2))        
        E=np.exp(-damp*omega*dt)
        K=omegad*dt
        C=E*np.cos(K)
        ac[1]=-2*C
        ac[2]=+E**2    
        
        return ac
    def rainflow(resp,bex):
        
        num=len(resp)
    
        y=np.zeros(num,'f')
        a=np.zeros(num,'f')
        B=np.zeros((num,4),'f')
    
        y=resp
        k=0
        a[0]=y[0]
    
        k=1
    
        for i in range (1,(num-1)):
            
            slope1=(  y[i]-y[i-1])
            slope2=(y[i+1]-y[i])
    
            if((slope1*slope2)<=0):
                a[k]=y[i]
                k+=1
                 
        last_a=k        
        hold=last_a        
        a[k]=y[num-1]
    
        mina=min(a)
        maxa=max(a)
    
        nmm=int(maxa-mina)+1
    
        n=0
        i=0
        j=1
    
        sum=0
        kv=0
    
        ymax=-1.0e+20
    
        aa=a.tolist()
    
        nkv=0
    
        ijk=0
        LLL=int(0.2*float(hold/2))
    
        while(1):
    
    
            Y=abs(aa[i]-aa[i+1])
            X=abs(aa[j]-aa[j+1])
    
    
            if(X>=Y and Y>0):		
                if(Y>ymax):
                    ymax=Y
             
                if(i==0):		
                    n=0
                    sum+=0.5
                    B[kv][3]=aa[i+1]
                    B[kv][2]=aa[i]
                    B[kv][1]=0.5
                    B[kv][0]=Y
                    kv+=1
                    aa.pop(i)
                    last_a-=1
                    i=0
                    j=1    
                
                else:      
                    sum+=1
                    B[kv][3]=aa[i+1]
                    B[kv][2]=aa[i]
                    B[kv][1]=1.
                    B[kv][0]=Y
                    kv+=1	
    
                    n=0
                
                    aa.pop(i+1)
                    aa.pop(i)                
                
                    i=0
                    j=1
    
                    last_a-=2
            
                    nkv+=1
    
    
                    ijk+=1  
    
                    if(ijk==LLL):
                        ijk=0               
            else:
                i+=1
                j+=1
                
            if((j+1)>last_a):
                break                
    
    
        for i in range (0,last_a):
            Y=(abs(aa[i]-aa[i+1]))
    
            if(Y>0):
                sum+=0.5
                B[kv][3]=aa[i+1]
                B[kv][2]=aa[i]
                B[kv][1]=0.5
                B[kv][0]=Y
        
                kv+=1
    
            damage=0
    
            for i in range (0,kv+1):
       
               Y=B[i][0]
               damage+=B[i][1]*((Y/2.)**bex)        
        
        
        return damage
    def accel_SRS(num_fn,fn,damp,dt):
        
        pos    = np.zeros(num_fn)
        neg    = np.zeros(num_fn)
        damage = np.zeros(num_fn)        
        bc     = np.zeros(3)  
        
        for j in range(0,num_fn):
            
            omega=2.*np.pi*fn[j]
            omegad=omega*np.sqrt(1.-(damp**2))
        #
        #  bc coefficients are applied to the excitation
            
            E=np.exp(-damp*omega*dt)
            K=omegad*dt
            C=E*np.cos(K)
            S=E*np.sin(K)
            Sp=S/K
    
            bc[0]=1.-Sp
            bc[1]=2.*(Sp-C)
            bc[2]=E**2-Sp
                        
            ac=a_coeff(omega,damp,dt);
    
      
            resp=lfilter(bc, ac, b, axis=-1, zi=None)            
    
            pos[j]= max(resp)
            neg[j]= abs(min(resp))    
            
            damage[j]=rainflow(resp,bex);
            
            ma=max(abs(resp))
            
            print(" %8.4g \t %8.4g \t %8.4g " %(fn[j],ma,damage[j]))
            
        return pos,neg,damage
    def rel_disp_SRS(num_fn,fn,damp,dt,irdu):
        
        rd_pos = np.zeros(num_fn)
        rd_neg = np.zeros(num_fn)
        damage = np.zeros(num_fn)        
        ac     = np.zeros(3)
        bc     = np.zeros(3)
                 
       
        for j in range(0,num_fn):
            
            omega=2.*np.pi*fn[j]
            omegad=omega*np.sqrt(1.-(damp**2))            
            
            E =np.exp(  -damp*omega*dt)
            E2=np.exp(-2*damp*omega*dt)
             
            K=omegad*dt
            C=E*np.cos(K)
            S=E*np.sin(K)
            
            Omr=(omega/omegad)
            Omt=omega*dt
            
            P=2*damp**2-1
            
            b00=2*damp*(C-1)
            b01=S*Omr*P
            b02=Omt
            
            b10=-2*Omt*C
            b11= 2*damp*(1-E2)
            b12=-2*b01   
    
            b20=(2*damp+Omt)*E2
            b21= b01
            b22=-2*damp*C               
            
            bc[0]=b00+b01+b02
            bc[1]=b10+b11+b12
            bc[2]=b20+b21+b22
            
            bc=-bc/(omega**3*dt)
                        
            ac=a_coeff(omega,damp,dt)
            
            resp=lfilter(bc, ac, b, axis=-1, zi=None)
            
            if(irdu==1):
                resp*=386
            else:
                resp*=9.81*1000
    
            rd_pos[j]= max(resp)
            rd_neg[j]= abs(min(resp))   
            
            damage[j]=rainflow(resp,bex);
            
            ma=max(abs(resp))
            
            #print(" %8.4g \t %8.4g \t %8.4g " %(fn[j],ma,damage[j]))
            
        return rd_pos,rd_neg,damage   
    if(iom==1):
        fp=np.zeros(num_fn);
        x_pos,x_neg,damage=accel_SRS(num_fn,fn,damp,dt);
    if(iom==2):
        rd_pos,rd_neg,damage=rel_disp_SRS(num_fn,fn,damp,dt,irdu);
        
    return fn,damage

In [ ]:
import numpy as np
from numpy.random import normal
import matplotlib.pyplot as plt
from scipy import stats,signal
from scipy.optimize import curve_fit
from velox_correction import velox_correction
from tompy import signal_stats,sample_rate_check,small
from scipy.signal import welch

In [ ]:
def psd2th_tom(freq,psd,tmax):
    def EnterPSD(freq,psd):
    
        a,b,num =freq,psd,len(psd);
    
        a=np.array(a)
        b=np.array(b)
        
    
        nm1=num-1
    
        slope =np.zeros(nm1,'f')
    
    
        ra=0
    
        for i in range (0,nm1):
    
            s=np.log(b[i+1]/b[i])/np.log(a[i+1]/a[i])
            
            slope[i]=s
    
            if s < -1.0001 or s > -0.9999:
                ra+= ( b[i+1] * a[i+1]- b[i]*a[i])/( s+1.)
            else:
                ra+= b[i]*a[i]*np.log( a[i+1]/a[i])
    
        omega=2*np.pi*a
    
        bv=np.zeros(num,'f') 
        bd=np.zeros(num,'f') 
            
        for i in range (0,num):         
            bv[i]=b[i]/omega[i]**2
         
        bv=bv*386**2
        rv=0
    
        for i in range (0,nm1):
    
            s=np.log(bv[i+1]/bv[i])/np.log(a[i+1]/a[i])
    
            if s < -1.0001 or s > -0.9999:
                rv+= ( bv[i+1] * a[i+1]- bv[i]*a[i])/( s+1.)
            else:
                rv+= bv[i]*a[i]*log( a[i+1]/a[i])         
             
            
        for i in range (0,num):         
            bd[i]=bv[i]/omega[i]**2
         
        rd=0
    
        for i in range (0,nm1):
    
            s=np.log(bd[i+1]/bd[i])/np.log(a[i+1]/a[i])
    
            if s < -1.0001 or s > -0.9999:
                rd+= ( bd[i+1] * a[i+1]- bd[i]*a[i])/( s+1.)
            else:
                rd+= bd[i]*a[i]*log( a[i+1]/a[i])         
    
    
        rms=np.sqrt(ra)
        three_rms=3*rms
    
        grms=rms
    
        vrms=np.sqrt(rv)
        vthree_rms=3*vrms
    
        drms=np.sqrt(rd)
        dthree_rms=3*drms
    
        return a,b,grms,num,slope     
    ########################################################################
    
    def magnitude_resolve(mmm,mH,Y):
    #
        mHm1=mH-1
        z=np.zeros(mH,'f')
        mag_seg=np.zeros(mH,'f')
    #
    #     for i in range (0,mH):
    #       z[i]=sqrt(Y.real[i]**2+Y.imag[i]**2)
    #
        z=abs(Y)/float(mmm)
    #
        mag_seg[0]=z[0]**2
    #
        mag_seg[1:mHm1]=((2*z[1:mHm1])**2)/2
    #
        return mag_seg
    
    ########################################################################
    
    def Hanning_initial(mmm):
        H=np.zeros(mmm,'f')
        tpi=2*np.pi
        alpha=np.linspace(0,tpi,mmm)
        ae=np.sqrt(8./3.)
        H=ae*0.5*(1.-np.cos(alpha))
        return H
    
    ########################################################################
    
    def psd_core(mmm,mH,maxf,NW,b):
        full=np.zeros(mH,'f')
    
        mag_seg=np.zeros(mH,'f')
        amp_seg=np.zeros(mmm,'f')
        den=df*(2*NW-1)
    
        nov=0
        for ijk in range (1,2*NW):
    
            amp_seg[0:mmm]=b[(0+nov):(mmm+nov)]
    
            nov=nov+int(mmm/2)
    
            mean = sum(amp_seg)/float(mmm)
            amp_seg-=mean
    
            amp_seg*=H
    
            Y = np.fft.fft(amp_seg)
    
            mag_seg = magnitude_resolve(mmm,mH,Y)
    
            full+=mag_seg
    
        
        full/=den
    
        ms=sum(full)
    
        return full,ms
    
    ########################################################################
    tpi=2*np.pi
    
    freq_spec,amp_spec,rms,num,slope = EnterPSD(freq,psd)
    
    nm1 = num - 1
    LS = nm1
    
    three_rms = 3 * rms
    fmax = max(freq_spec)
    
    sr = fmax * 20.
    dt = 1 / sr
    
    spec_grms = rms
    
    np1 = int(np.ceil(tmax / dt))
    np3 = int(3 * np1)
    
    mu = 0
    sigma = 1
    # Generate white noise
    white_noise = normal(mu, sigma, np3)
    num_fft=2
    
    while(num_fft<np1):
        num_fft*=2
    
    N = num_fft
    df = 1. / (N * dt)
    
    m2 = int(num_fft / 2)
    
    fft_freq = np.linspace(0, (m2 - 1) * df, m2)
    fft_freq2 = np.linspace(0, (num_fft - 1) * df, num_fft)
    
    spec = np.zeros(m2, float)
    sq_spec = np.zeros(m2, float)
    
    js = 0
    # Interpolate specification
    
    if(fft_freq[0]<=0):
        fft_freq[0]=0.5*fft_freq[1]        
            
    x=np.log10(fft_freq)    
    xp=np.log10(freq_spec)    
    yp=np.log10(amp_spec)
            
    y=np.interp(x, xp, yp, left=-10, right=-10)
    
    sq_spec=np.sqrt(10**y)
    # add option for sine tones later
    
    # Calculating FFT 
    
    white_noise_trunc = white_noise[0:num_fft]
    
    
    Y = np.zeros(num_fft, complex)
    
    YF = np.fft.fft(white_noise_trunc)
    
    # Apply spec 
    
    
    YFn=YF[0:m2]
    
    Y[0:m2]=sq_spec*YFn
    
    Y[0]=0.
    # Make symmetric
    
    for j in range(1, m2):
        Y[num_fft - j] = complex(Y[j].real, -Y[j].imag)
    # Calculating inverse FFT 
    
    YI = np.fft.ifft(Y);
    # YIR
    
    YIR = YI.real
    # psd_th_m2
    
    psd_th_m2 = YIR
    
    nL = m2
    # psd_th
    
    psd_th = YIR[0:np1]
    
    np1 = len(psd_th)
    
    TT = np.linspace(0, (np1 - 1) * dt, np1)
    
    stddev = np.std(psd_th)
    
    psd_th *= (spec_grms / stddev)
    # check psd
    a=TT
    b=psd_th
    
    num=len(a)
    
    sr,dt,mean,sd,rms,skew,kurtosis,dur=signal_stats(a,b,num)
    
    sr,dt=sample_rate_check(a,b,num,sr,dt)
    
    # Remove mean:  1=yes  2=no 
    
    mr_choice = 1
    
    # elect Window: 1=Rectangular 2=Hanning 
    
    h_choice = 2
    
    n=num
    
    ss=np.zeros(n)
    seg=np.zeros(n,'f')
    i_seg=np.zeros(n)
    
    NC=0
    for i in range (0,1000):
        nmp = 2**(i-1)
        if(nmp <= n ):
            ss[i] = 2**(i-1)
            seg[i] =float(n)/float(ss[i])
            i_seg[i] = int(seg[i])
            NC=NC+1
        else:
            break
    #print (' ')
    #print (' Number of   Samples per   Time per        df    ')
    #print (' Segments     Segment      Segment(sec)   (Hz)   dof')
    
    for i in range (1,NC+1):
        j=NC+1-i
        if j>0:
            if( i_seg[j]>0 ):
                tseg=dt*ss[j]
                ddf=1./tseg
                #print ('%8d \t %8d \t %10.3g  %10.3g    %d' \
                                             #%(i_seg[j],ss[j],tseg,ddf,2*i_seg[j]))
        if(i==12):
            break
    ijk=0
    while ijk==0:
        ##################### Choose the Number of Segments ##########################
        i = 6; 
        j=NC+1-i;
        
        s=i_seg[j]
        NW = int(s)
        for j in range (0,len(i_seg)):
            if NW==i_seg[j]:
                ijk=1
                break
    # check
    
    mmm = 2**int(np.log(float(n)/float(NW))/np.log(2))
    
    df=1./(mmm*dt)
    
    # begin overlap
    
    mH=((mmm/2)-1)
    
    mH = int(mH);
    
    #print (" ")
    #print ("     number of segments   NW= %d " %NW)
    #print ("       samples/segments  mmm= %d " %mmm)
    #print (" half samples/segment-1   mH=%d  " %mH)
    #print (" ")
    #print ("        df=%6.3f Hz" %df)
    
    maxf=(mH-1)*df
    
    H=Hanning_initial(mmm)
    
    freq=np.zeros(mH,'f')
    freq=np.linspace(0,maxf,mH)
    
    delta=freq_spec[0]/30
    
    nnt=3
    
    # Velocity correction
    
    
    for kvn in range (0,nnt):
    
        acc,velox,dispx=velox_correction(psd_th,dt,freq_spec[0])
    
        ratio = spec_grms/np.std(psd_th)
    
        psd_th*= ratio
        velox*= ratio
        dispx*= ratio
        
        full,ms=psd_core(mmm,mH,maxf,NW,psd_th)
    
        MK=len(psd_th)
        tim=np.linspace(0,MK*dt,MK)
        

    return TT, psd_th, velox, dispx

In [ ]:
def blwn_test_td(fmin,fmax,bwp,loc,z,mm,amp,ovl,dur):  # time domain
    # Generate Input PSD
    freq_spec = np.logspace(np.log10(fmin),np.log10(fmax),5000); psd_spec = np.zeros(5000); 
    for i in range(5000):
        psd_spec[i]= blwn(freq_spec[i],fmin,fmax,bwp,loc);
    # Band Split
    fi,pi = freq_spec,psd_spec;
    dfi,dpi = band_split(fi,pi,2);
    if ovl > 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
        dfi,dpi = bs_overlap(dfi,dpi,ovl); # overlap
        
    # Generate Time History
    t1,th1, _, _ = psd2th_tom(fi,pi,dur);
    t2,th2, _, _ = psd2th_tom(dfi[0],dpi[0],dur);
    t3,th3, _, _ = psd2th_tom(dfi[1],dpi[1],dur);
    # Calculate PSD of TH
    f_syn1,p_syn1 = th2psd(t1,th1,2**12);
    f_syn2,p_syn2 = th2psd(t2,th2,2**12);
    f_syn3,p_syn3 = th2psd(t3,th3,2**12);
    # Calculate FDS
    f1,f2,oct,iom,irdu,bex = fmin,fmax,1./18.,2,1,4;
    fn,D = th2fds_tom(t1,th1,f1,f2,oct,iom,irdu,bex)
    fn,D1= th2fds_tom(t2,th2,f1,f2,oct,iom,irdu,bex)
    fn,D2 = th2fds_tom(t3,th3,f1,f2,oct,iom,irdu,bex)

    # Plot
    plt.figure(figsize=(16,16));
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
    plt.subplot(321);
    plt.plot(dfi[0],dpi[0],label='band1'); plt.plot(dfi[1],dpi[1],label='band2'); plt.legend();
    plt.ylim(max(dpi[0])/100,max(dpi[0])*100);
    #plt.xscale('log'); 
    plt.yscale('log'); plt.xlim(fmin,fmax); plt.title('Input PSD(BW=%.1f, LOC=%.1f)'%(bwp,loc)); plt.xlabel('Freq(Hz)'); plt.ylabel('PSD(g^2/Hz)');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(322);
    plt.plot(fn,D,label='original'); plt.plot(fn,D1,label='band1'); plt.plot(fn,D2,label='band2'); 
    #plt.xscale('log'); 
    plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(323);
    if ovl == 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
    ylim1 = min(D[(fn>xlim1)&(fn<xlim2)])*0.1; 
    ylim2 = max(D[(fn>xlim1)&(fn<xlim2)])*10; 
    plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1,'r--',label='band1'); plt.plot(fn,D2,'b--',label='band2'); 
    plt.plot(fn,D1+D2,'g--',label='split(sum)');  plt.yscale('log');
    plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    plt.title('FDS, Near Band Split Freq');
    plt.grid(b=True, which='major', linestyle='-');
    plt.subplot(324);
    plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1+D2,'r--',label='split(sum)');  
    #plt.xscale('log'); 
    plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(325);
    rD = D/(D1+D2);
    plt.plot(fn,rD,'k-',label='original/split(sum)');
    plt.legend(); plt.xlim(fmin,fmax);
    #plt.xscale('log'); 
    plt.title('FDS Ratio(original/split(sum))'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(326);
    plt.plot(fn,rD,'k-',label='original/split(sum)');
    ylim1 = min(rD[(fn>xlim1)&(fn<xlim2)])*0.5; ylim2 = max(rD[(fn>xlim1)&(fn<xlim2)])*2;
    plt.grid(which='both'); plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    plt.title('FDS Ratio(original/split(sum)), Near Band Split Freq'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');\
    plt.grid(b=True, which='both', linestyle='-');
    plt.savefig('blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_td.png'%(bwp,loc,z,mm,amp,ovl), dpi=300);  
    plt.close();
    
    fileO(fn,rD,'blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_td_rD.txt'%(bwp,loc,z,mm,amp,ovl));
    fileO(fn,D,'blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_td_FDS1.txt'%(bwp,loc,z,mm,amp,ovl));
    fileO(fn,D1,'blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_td_FDS2.txt'%(bwp,loc,z,mm,amp,ovl));
    fileO(fn,D2,'blwn_%.1fb_%.1fl_%.2fz_%dm_%.2fa_%.2fo_td_FDS3.txt'%(bwp,loc,z,mm,amp,ovl));

In [ ]:
def unimodal_test_td(fmin,fmax,bwp,loc,fn1,zeta1,z,mm,amp,ovl,dur):  # time domain
    # Generate Input PSD
    freq_spec = np.logspace(np.log10(fmin),np.log10(fmax),5000); psd_spec = np.zeros(5000); 
    for i in range(5000):
        psd_spec[i]= unimodal(freq_spec[i],fmin,fmax,bwp,loc,fn1,zeta1);
    # Band Split
    fi,pi = freq_spec,psd_spec;
    dfi,dpi = band_split(fi,pi,2);
    if ovl > 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
        dfi,dpi = bs_overlap(dfi,dpi,ovl); # overlap    

    # Generate Time History
    t1,th1, _, _ = psd2th_tom(fi,pi,dur);
    t2,th2, _, _ = psd2th_tom(dfi[0],dpi[0],dur);
    t3,th3, _, _ = psd2th_tom(dfi[1],dpi[1],dur);
    # Calculate PSD of TH
    f_syn1,p_syn1 = th2psd(t1,th1,2**12);
    f_syn2,p_syn2 = th2psd(t2,th2,2**12);
    f_syn3,p_syn3 = th2psd(t3,th3,2**12);
    # Calculate FDS
    f1,f2,oct,iom,irdu,bex = fmin,fmax,1./18.,2,1,4;
    fn,D = th2fds_tom(t1,th1,f1,f2,oct,iom,irdu,bex)
    fn,D1= th2fds_tom(t2,th2,f1,f2,oct,iom,irdu,bex)
    fn,D2 = th2fds_tom(t3,th3,f1,f2,oct,iom,irdu,bex)
        
    # Plot
    plt.figure(figsize=(16,16));
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
    plt.subplot(321);
    plt.plot(dfi[0],dpi[0],label='band1'); plt.plot(dfi[1],dpi[1],label='band2'); plt.legend();
    #plt.xscale('log'); 
    plt.yscale('log'); plt.xlim(fmin,fmax); plt.title('Input PSD(BW=%.1f, LOC=%.1f, fn1 = %d, zeta1 = %.2f)'%(bwp,loc,fn1,zeta1)); plt.xlabel('Freq(Hz)'); plt.ylabel('PSD(g^2/Hz)');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(322);
    plt.plot(fn,D,label='original'); plt.plot(fn,D1,label='band1'); plt.plot(fn,D2,label='band2'); 
    #plt.xscale('log'); 
    plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(323);
    if ovl == 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
    ylim1 = min(D[(fn>xlim1)&(fn<xlim2)])*0.1; 
    ylim2 = max(D[(fn>xlim1)&(fn<xlim2)])*10; 
    plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1,'r--',label='band1'); plt.plot(fn,D2,'b--',label='band2'); 
    plt.plot(fn,D1+D2,'g--',label='split(sum)');  plt.yscale('log');
    plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    plt.title('FDS, Near Band Split Freq');
    plt.grid(b=True, which='major', linestyle='-');
    plt.subplot(324);
    plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1+D2,'r--',label='split(sum)');  
    #plt.xscale('log');
    plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(325);
    rD = D/(D1+D2);
    plt.plot(fn,rD,'k-',label='original/split(sum)');
    plt.legend(); plt.xlim(fmin,fmax);
    #plt.xscale('log'); 
    plt.title('FDS Ratio(original/split(sum))'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(326);
    plt.plot(fn,rD,'k-',label='original/split(sum)');
    ylim1 = min(rD[(fn>xlim1)&(fn<xlim2)])*0.5; ylim2 = max(rD[(fn>xlim1)&(fn<xlim2)])*2;
    plt.grid(which='both'); plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    plt.title('FDS Ratio(original/split(sum)), Near Band Split Freq'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');\
    plt.grid(b=True, which='both', linestyle='-');
    plt.savefig('uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_td.png'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl), dpi=300);   # Bandwidth, Location, 1% damping FDS, m = 4
    plt.close();
    
    fileO(fn,rD,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_td_rD.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
    fileO(fn,D,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_td_FDS1.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
    fileO(fn,D1,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_td_FDS2.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));
    fileO(fn,D2,'fds/unimodal/uni_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2fz_%dm_%.2fa_%.2fo_td_FDS3.txt'%(bwp,loc,fn1,zeta1,z,mm,amp,ovl));

In [ ]:
def bimodal_test_td(fmin,fmax,bwp,loc,fn1,zeta1,fn2,zeta2,ratio,z,mm,amp,ovl,dur): # time domain
    # Generate Input PSD
    freq_spec = np.logspace(np.log10(fmin),np.log10(fmax),5000); psd_spec = np.zeros(5000); 
    for i in range(5000):
        psd_spec[i]= bimodal(freq_spec[i],fmin,fmax,bwp,loc,fn1,zeta1,fn2,zeta2,ratio);
    # Band Split
    fi,pi = freq_spec,psd_spec;
    dfi,dpi = band_split(fi,pi,2);
    if ovl > 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
        dfi,dpi = bs_overlap(dfi,dpi,ovl); # overlap  

    # Generate Time History
    t1,th1, _, _ = psd2th_tom(fi,pi,dur);
    t2,th2, _, _ = psd2th_tom(dfi[0],dpi[0],dur);
    t3,th3, _, _ = psd2th_tom(dfi[1],dpi[1],dur);
    # Calculate PSD of TH
    f_syn1,p_syn1 = th2psd(t1,th1,2**12);
    f_syn2,p_syn2 = th2psd(t2,th2,2**12);
    f_syn3,p_syn3 = th2psd(t3,th3,2**12);
    # Calculate FDS
    f1,f2,oct,iom,irdu,bex = fmin,fmax,1./18.,2,1,4;
    fn,D = th2fds_tom(t1,th1,f1,f2,oct,iom,irdu,bex)
    fn,D1= th2fds_tom(t2,th2,f1,f2,oct,iom,irdu,bex)
    fn,D2 = th2fds_tom(t3,th3,f1,f2,oct,iom,irdu,bex)

    # Plot
    plt.figure(figsize=(16,16));
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=0.2, hspace=0.4)
    plt.subplot(321);
    plt.plot(dfi[0],dpi[0],label='band1'); plt.plot(dfi[1],dpi[1],label='band2'); plt.legend();
    #plt.xscale('log'); 
    plt.yscale('log'); plt.xlim(fmin,fmax); plt.title('Input PSD(BW=%.1f, LOC=%.1f, fn1 = %d, zeta1 = %.2f, fn2 = %d, zeta2 = %.2f, ratio = %.2f)'%(bwp,loc,fn1,zeta1,fn2,zeta2,ratio)); plt.xlabel('Freq(Hz)'); plt.ylabel('PSD(g^2/Hz)');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(322);
    plt.plot(fn,D,label='original'); plt.plot(fn,D1,label='band1'); plt.plot(fn,D2,label='band2'); 
    #plt.xscale('log'); 
    plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(323);
    if ovl == 0:
        xlim1 = max(dfi[0])*0.9; xlim2 = min(dfi[1])*1.1;
    ylim1 = min(D[(fn>xlim1)&(fn<xlim2)])*0.1; 
    ylim2 = max(D[(fn>xlim1)&(fn<xlim2)])*10; 
    plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1,'r--',label='band1'); plt.plot(fn,D2,'b--',label='band2'); 
    plt.plot(fn,D1+D2,'g--',label='split(sum)');  plt.yscale('log');
    plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    plt.title('FDS, Near Band Split Freq');
    plt.grid(b=True, which='major', linestyle='-');
    plt.subplot(324);
    plt.plot(fn,D,'k-',label='original'); plt.plot(fn,D1+D2,'r--',label='split(sum)');  
    #plt.xscale('log'); 
    plt.yscale('log'); plt.legend(); plt.xlim(fmin,fmax);
    plt.title('FDS'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(325);
    rD = D/(D1+D2);
    plt.plot(fn,rD,'k-',label='original/split(sum)');
    plt.legend(); plt.xlim(fmin,fmax);
    #plt.xscale('log'); 
    plt.title('FDS Ratio(original/split(sum))'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');
    plt.grid(b=True, which='both', linestyle='-');
    plt.subplot(326);
    plt.plot(fn,rD,'k-',label='original/split(sum)');
    ylim1 = min(rD[(fn>xlim1)&(fn<xlim2)])*0.5; ylim2 = max(rD[(fn>xlim1)&(fn<xlim2)])*2;
    plt.grid(which='both'); plt.legend(); plt.xlim(xlim1,xlim2); plt.ylim(ylim1,ylim2);
    plt.title('FDS Ratio(original/split(sum)), Near Band Split Freq'); plt.xlabel('Natural Freq(Hz)'); plt.ylabel('Pseudo Fatigue Damage Ratio');\
    plt.grid(b=True, which='both', linestyle='-');
    plt.savefig('bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_td.png'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl), dpi=300);   # Bandwidth, Location, 1% damping FDS, m = 4
    plt.close();
    
    fileO(fn,rD,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_td_rD.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
    fileO(fn,D,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_td_FDS1.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
    fileO(fn,D1,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_td_FDS2.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
    fileO(fn,D2,'fds/bimodal/bi_%.1fb_%.1fl_%.2ff1_%.2fz1_%.2ff2_%.2fz2_%.2fz_%dm_%.2fa_%.2fo_td_FDS3.txt'%(bwp,loc,fn1,zeta1,fn2,zeta2,z,mm,amp,ovl));
